In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:13 https://

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Advertising Spend").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/AdvertisingSpend.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("AdvertisingSpend.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|              Type|            Source|              2012|              2013|              2014|              2015|              2016|              2017|              2018|              2019|              2020|              2021|              2022|              2023|              2024|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          Total TV|          Total TV|  	60,914,800,000 |  	62,895,700,000 |  	64,401,700,000 |  	65,382,100,000 |  	67,440,100,000 |  	66

In [13]:
def dollars_to_millions(dollars):
  dollars = dollars.replace("\t", "")
  dollars = dollars.replace(",", "")
  dollars = dollars.replace(" ", "")
  dollars = float(dollars)
  millions = dollars / 1000000
  return millions

In [14]:
spark.udf.register("dollars_to_millions", dollars_to_millions)

<function __main__.dollars_to_millions>

In [15]:
df.createOrReplaceTempView("dollars_to_millions")

In [16]:
temp_df = spark.sql("SELECT source, dollars_to_millions(`2020`), dollars_to_millions(`2021`), dollars_to_millions(`2022`), dollars_to_millions(`2023`), dollars_to_millions(`2024`) FROM dollars_to_millions")
temp_df.show()

+------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
|            source|dollars_to_millions(2020)|dollars_to_millions(2021)|dollars_to_millions(2022)|dollars_to_millions(2023)|dollars_to_millions(2024)|
+------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
|          Total TV|                  52560.3|                  53182.8|                  51431.6|                  52055.3|                  52196.7|
|       National TV|                  38024.8|                  40138.7|                  40227.3|                  40285.0|                  40310.1|
|          Local TV|                  14535.5|                  13044.1|                  11204.3|                  11770.2|                  11886.6|
|       Total Radio|                  12085.4|                  11179.5|                  1095